In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from pandas.plotting import table
import plotly.express as px
import matplotlib.ticker as mtick

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import statsmodels.api as sm
import numpy as np

In [ ]:
q_data= pd.read_csv("qualitative_data_msba.csv")
t_data=pd.read_csv("time_series_data_msba.csv")

In [ ]:
q_data.head()

In [ ]:
t_data.head()

In [ ]:
q_data= q_data.drop("Unnamed: 0", axis=1)
t_data= t_data.drop("Unnamed: 0", axis=1)

In [ ]:
#dropping columns with only one and same unique value as they wont contribute to model development
q_data.drop(['front_door_count','godfather_s_pizza','car_wash','ev_charging','non_24_hour','self_check_out','diesel'], axis=1,inplace=True)

In [ ]:
q_data.shape

In [ ]:
cat_col= q_data.select_dtypes(include="object").columns.tolist()
cat_col

In [ ]:
#checking columns with na values and adding them all to a list
columns_with_na = q_data.columns[q_data.isna().any()].tolist()
columns_with_na

In [ ]:
#replacing NAs with None
for column in columns_with_na:
    #mode_value = q_data[column].mode()[0]
    q_data[column].fillna("None", inplace=True)

In [ ]:
#verify that there are no na values anymore
q_data.columns[q_data.isna().any()].tolist()

In [ ]:
cleaned_q_data= q_data

In [ ]:
#merge
df_merged_clean= pd.merge(t_data, cleaned_q_data, on='site_id_msba', how='left')

In [ ]:
df_merged_clean['calendar_information.holiday'] = list(map(lambda x: (
    1 if x == 'None'
    else 0), df_merged_clean['calendar_information.holiday']))

In [ ]:
pd.set_option('display.max_columns', None)

df_merged_clean.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['calendar_information.type_of_day', 'ethanol_free',
'hi_flow_lanes', 'rv_lanes', 'hi_flow_rv_lanes', 'def', 'cat_scales', 'rv_dumps', 'propane', 
                       'traditional_forecourt_stack_type', 'rv_lanes_layout', 'rv_lanes_stack_type', 'hi_flow_lanes_layout'
                       ,'hi_flow_lanes_stack_type', 'hi_flow_rv_lanes_stack_type']

# # Initialize the LabelEncoder
label_encoder = LabelEncoder()

# # Apply label encoding to each of the specified categorical columns
for column in categorical_columns:
    df_merged_clean[column] = label_encoder.fit_transform(df_merged_clean[column])

In [ ]:
## OHE
df_encoded = pd.get_dummies(df_merged_clean, columns=['traditional_forecourt_layout', 'hi_flow_rv_lanes_layout'])

In [ ]:
pd.set_option('display.max_columns', None)

df_encoded.head()

In [ ]:
df_encoded.shape

In [ ]:
df_encoded = df_encoded.sort_values(by=['calendar.calendar_day_date'], ascending=True)

In [ ]:
pd.set_option('display.max_columns', None)

df_encoded.head()

In [ ]:
df_encoded.to_csv('unleaded1.csv', index=False)

In [ ]:
df_encoded= pd.read_csv('unleaded1.csv')

In [ ]:
df_encoded.head()

In [ ]:
df_encoded_train= df_encoded.loc[0:13000]
df_encoded_test= df_encoded.loc[13001:]

In [ ]:
df_encoded_train.tail()


In [ ]:
df_encoded_test.head()

In [ ]:
#Plot the unleaded sales of Store: site_id_msba=  24535

df2 = df_encoded_train[df_encoded_train.site_id_msba==  24535]

In [ ]:
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Unleaded Sales')
plt.plot(df2['unleaded'])
plt.title('site_id_msba=24535  Unleaded Sales')
plt.show()

# Test for stationarity
Stationarity check is important in time series as we need to check what mathematical operations on the series make it predictable for future. For stationary time series the mean and variance is constant, so it is more predictable compared to non stationary time series. One of the test for stationarity is Augmented Dickey Fuller Test (ADF for short)

In [ ]:
#Test for stationarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)
    
    print("Results of Dickey Fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)

In [ ]:
test_stationarity(df2.unleaded)

##### Based on the low p-value we observe that the time series is stationary

In [ ]:
result = seasonal_decompose(df2.unleaded, model='multiplicative', period=1)
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(16, 9)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
df_log = np.log(df2.unleaded)
moving_avg = df_log.rolling(12).mean()
std_dev = df_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

In [ ]:
#split data into train and training set
train_data, test_data = df_log[3:int(len(df_log)*0.9)], df_log[int(len(df_log)*0.9):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Unleaded Prices')
plt.plot(df_log, 'green', label='Train data')
plt.plot(test_data, 'blue', label='Test data')
plt.legend()

In [ ]:
df3 = df_encoded_train[['calendar_information.type_of_day', 'ethanol_free',
'hi_flow_lanes', 'rv_lanes', 'hi_flow_rv_lanes', 'def', 'cat_scales', 'rv_dumps', 'propane', 
                       'traditional_forecourt_stack_type', 'rv_lanes_layout', 'rv_lanes_stack_type', 'hi_flow_lanes_layout'
                       ,'hi_flow_lanes_stack_type', 'hi_flow_rv_lanes_stack_type', 'traditional_forecourt_layout_In-Line', 
 'traditional_forecourt_layout_Stack', 'hi_flow_rv_lanes_layout_Combo', 'hi_flow_rv_lanes_layout_In-Line', 'hi_flow_rv_lanes_layout_None',
 'hi_flow_rv_lanes_layout_Stack']]
model_autoARIMA = auto_arima(df_encoded_train.unleaded, start_p=0, start_q=0,
                      test='adf',       # use adftest to find             optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True,
                      exogenous= df3)
print(model_autoARIMA.summary())

## The best model is ARIMA(3,0,0) which means we are describing sales response variable (Y) by combining a 3rd order Auto-Regressive model and a 0th order Moving Average model.

In [ ]:
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

## Pick the best model for ARIMAX

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model= SARIMAX(df_encoded_train.unleaded, 
 exog=df3,
 order=(2,0,0),
 enforce_invertibility=False, enforce_stationarity=False)

In [ ]:
df_test2 = df_encoded_test[df_encoded_test.site_id_msba == 24535]

In [ ]:
df_test2.head()

In [ ]:
df_test2.shape

In [ ]:
df_test3 = df_test2[['calendar_information.type_of_day', 'ethanol_free',
'hi_flow_lanes', 'rv_lanes', 'hi_flow_rv_lanes', 'def', 'cat_scales', 'rv_dumps', 'propane', 
                       'traditional_forecourt_stack_type', 'rv_lanes_layout', 'rv_lanes_stack_type', 'hi_flow_lanes_layout'
                       ,'hi_flow_lanes_stack_type', 'hi_flow_rv_lanes_stack_type', 'traditional_forecourt_layout_In-Line', 
 'traditional_forecourt_layout_Stack', 'hi_flow_rv_lanes_layout_Combo', 'hi_flow_rv_lanes_layout_In-Line', 'hi_flow_rv_lanes_layout_None',
 'hi_flow_rv_lanes_layout_Stack']]

In [ ]:
results= model.fit()
forecast_1= results.forecast(steps=len(df_test2.site_id_msba), exog=df_test3)

In [ ]:
forec= list(forecast_1)

In [ ]:
forec

In [ ]:
train_data = df2.unleaded
test_data = forecast_1
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Unleaded sales')
plt.plot(train_data, 'green', label='Train data')
plt.plot(test_data, 'blue', label='Test data')
plt.legend()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA



# Load your sales data into a Pandas DataFrame
# Replace 'your_data.csv' with the actual path to your data file
df = pd.read_csv('unleaded1.csv')
df['date'] = pd.to_datetime(df['calendar.calendar_day_date'])
df.set_index('date', inplace=True)

# Visualize your sales data
plt.figure(figsize=(12, 6))
plt.plot(df['unleaded'])
plt.title('Sales Data')
plt.xlabel('Date')
plt.ylabel('Unleaded Sales')
plt.show()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# Fit an ARIMA model
p = 3  # Autoregressive order
d = 0  # Differencing order
q = 0  # Moving average order

model = ARIMA(df_encoded['unleaded'], order=(p, d, q))
model_fit = model.fit()

# Forecast the next 12 months
forecast_steps = 365
forecast = model_fit.forecast(steps=forecast_steps)

In [ ]:
forecast

In [ ]:
forecast_s= pd.Series(list(forecast))

In [ ]:
df6 = pd.DataFrame({'unleaded_sales_forecast': forecast_s}) 

In [ ]:
df6